In [1]:
import pandas as pd
import re
from underthesea import word_tokenize

def clean_special_characters(text):
    return re.sub(r'[<>/\[\]_\\\*\&\^\#\`\~\-]', '', text)

# Đọc hai file
df_true = pd.read_csv('real.csv')
df_fake = pd.read_csv('fake.csv')

# Nối 2 dataframe
df = pd.concat([df_true, df_fake], ignore_index=True)

# Loại bỏ cột 'source' nếu tồn tại
if 'source' in df.columns:
    df = df.drop(columns=['source'])

# Loại bỏ các dòng có giá trị thiếu
df = df.dropna()

df['title'] = df['title'].astype(str).str.strip()
df['content'] = df['content'].astype(str).str.strip()

df['title'] = df['title'].apply(clean_special_characters)
df['content'] = df['content'].apply(clean_special_characters)


# Chuẩn hóa cột label về chữ thường
df['label'] = df['label'].astype(str).str.lower()

# Chuyển thành nhãn nhị phân: true -> 1, false -> 0
df['label'] = df['label'].map({'true': 1, 'false': 0})

# (Tùy chọn) kiểm tra các giá trị không hợp lệ còn lại (nếu có)
invalid_labels = df[df['label'].isna()]
if not invalid_labels.empty:
    print("Cảnh báo: Có nhãn không hợp lệ:")
    print(invalid_labels['label'].unique())

df = df.dropna(subset=['title','content'])
df = df[df['content'].str.strip() != '']
df = df[df['title'].str.strip() != '']

# Xử lý stop word
with open("./stop_word/vietnamese-stopwords.txt", "r", encoding="utf-8") as f:
    stopwords = set(line.strip() for line in f.readlines())
    
def preprocess(text):
    # Tách từ và giữ lại dạng có dấu gạch dưới
    tokens = word_tokenize(text, format="text").split()
    # Loại bỏ stopword
    filtered = [word for word in tokens if word.lower() not in stopwords]
    return " ".join(filtered)

df['title'] = df['title'].apply(preprocess)
df['content'] = df['content'].apply(preprocess)

# Xuất ra file CSV
df.to_csv('merged_cleaned_dataset.csv', index=False)

print("Hoàn tất! Dataset đã được làm sạch và lưu vào 'merged_cleaned_dataset.csv'")


Hoàn tất! Dataset đã được làm sạch và lưu vào 'merged_cleaned_dataset.csv'
